In [2]:
import ijson
import polars as pl
import json

In [53]:
li = "/Users/mz/Downloads/133971298-1427616002_nyu-langone-long-island_standardcharges.csv"
ny = "/Users/mz/Downloads/133971298-1801992631_nyu-langone-tisch_standardcharges.csv"

In [ ]:
df = (pl.scan_csv(li, skip_rows=2, has_header=True, ignore_errors=True)
      .filter(pl.col("code|1| type") == "CPT", pl.col("code|1") == 92250)
).collect()

In [55]:
df.head()

code|1,code|1| type,code|2,code|2| type,code|3,code|3| type,Description,standard_charge|gross,standard_charge|discounted_cash,setting,drug_type_of_measurement,drug_unit_of_measurement,modifiers,standard_charge|1199|LOCAL 1199 NYU MEMBER CHOICE 1000|negotiated_percentage,standard_charge|1199|LOCAL 1199 NYU MEMBER CHOICE 1000|methodology,additional_payer_notes|1199|LOCAL 1199 NYU MEMBER CHOICE 1000,standard_charge|1199|LOCAL 1199 NYU MEMBER CHOICE 1000|negotiated_dollar,standard_charge|1199|LOCAL 1199 NYU MEMBER CHOICE 1000|negotiated_algorithm,estimated_amount|1199|LOCAL 1199 NYU MEMBER CHOICE 1000,standard_charge|1199|LOCAL 1199 PPO 1001|negotiated_percentage,standard_charge|1199|LOCAL 1199 PPO 1001|methodology,additional_payer_notes|1199|LOCAL 1199 PPO 1001,standard_charge|1199|LOCAL 1199 PPO 1001|negotiated_dollar,standard_charge|1199|LOCAL 1199 PPO 1001|negotiated_algorithm,estimated_amount|1199|LOCAL 1199 PPO 1001,standard_charge|First Health|FIRST HEALTH STRATEGIES 1002|negotiated_percentage,standard_charge|First Health|FIRST HEALTH STRATEGIES 1002|methodology,additional_payer_notes|First Health|FIRST HEALTH STRATEGIES 1002,standard_charge|First Health|FIRST HEALTH STRATEGIES 1002|negotiated_dollar,standard_charge|First Health|FIRST HEALTH STRATEGIES 1002|negotiated_algorithm,estimated_amount|First Health|FIRST HEALTH STRATEGIES 1002,standard_charge|Aetna|AETNA HMO 1005|negotiated_percentage,standard_charge|Aetna|AETNA HMO 1005|methodology,additional_payer_notes|Aetna|AETNA HMO 1005,standard_charge|Aetna|AETNA HMO 1005|negotiated_dollar,standard_charge|Aetna|AETNA HMO 1005|negotiated_algorithm,estimated_amount|Aetna|AETNA HMO 1005,…,additional_payer_notes|HIP|EMBLEM ESSENTIAL 1AND2 (COHORT 2 HCP) 3906,standard_charge|HIP|EMBLEM ESSENTIAL 1AND2 (COHORT 2 HCP) 3906|negotiated_dollar,standard_charge|HIP|EMBLEM ESSENTIAL 1AND2 (COHORT 2 HCP) 3906|negotiated_algorithm,estimated_amount|HIP|EMBLEM ESSENTIAL 1AND2 (COHORT 2 HCP) 3906,standard_charge|UHC|UNITED HEALTHCARE CHOICE (NY POWER AUTHORITY) 3911|negotiated_percentage,standard_charge|UHC|UNITED HEALTHCARE CHOICE (NY POWER AUTHORITY) 3911|methodology,additional_payer_notes|UHC|UNITED HEALTHCARE CHOICE (NY POWER AUTHORITY) 3911,standard_charge|UHC|UNITED HEALTHCARE CHOICE (NY POWER AUTHORITY) 3911|negotiated_dollar,standard_charge|UHC|UNITED HEALTHCARE CHOICE (NY POWER AUTHORITY) 3911|negotiated_algorithm,estimated_amount|UHC|UNITED HEALTHCARE CHOICE (NY POWER AUTHORITY) 3911,standard_charge|UHC|GOLDEN RULE INSURANCE 3914|negotiated_percentage,standard_charge|UHC|GOLDEN RULE INSURANCE 3914|methodology,additional_payer_notes|UHC|GOLDEN RULE INSURANCE 3914,standard_charge|UHC|GOLDEN RULE INSURANCE 3914|negotiated_dollar,standard_charge|UHC|GOLDEN RULE INSURANCE 3914|negotiated_algorithm,estimated_amount|UHC|GOLDEN RULE INSURANCE 3914,standard_charge|Medicare|BCBS HEALTHPLUS MEDIBLUE DUAL 3928|negotiated_percentage,standard_charge|Medicare|BCBS HEALTHPLUS MEDIBLUE DUAL 3928|methodology,additional_payer_notes|Medicare|BCBS HEALTHPLUS MEDIBLUE DUAL 3928,standard_charge|Medicare|BCBS HEALTHPLUS MEDIBLUE DUAL 3928|negotiated_dollar,standard_charge|Medicare|BCBS HEALTHPLUS MEDIBLUE DUAL 3928|negotiated_algorithm,estimated_amount|Medicare|BCBS HEALTHPLUS MEDIBLUE DUAL 3928,standard_charge|Medicare|CIGNA MEDICARE ADVANTAGE 3985|negotiated_percentage,standard_charge|Medicare|CIGNA MEDICARE ADVANTAGE 3985|methodology,additional_payer_notes|Medicare|CIGNA MEDICARE ADVANTAGE 3985,standard_charge|Medicare|CIGNA MEDICARE ADVANTAGE 3985|negotiated_dollar,standard_charge|Medicare|CIGNA MEDICARE ADVANTAGE 3985|negotiated_algorithm,estimated_amount|Medicare|CIGNA MEDICARE ADVANTAGE 3985,standard_charge|BCBS|ANTHEM BCBS CT PPO 4127|negotiated_percentage,standard_charge|BCBS|ANTHEM BCBS CT PPO 4127|methodology,additional_payer_notes|BCBS|ANTHEM BCBS CT PPO 4127,standard_charge|BCBS|ANTHEM BCBS CT PPO 4127|negotiated_dollar,standard_charge|BCBS|ANTHEM BCBS CT PPO 4127|negotiated_algorithm,estimate

In [58]:
(df.melt(id_vars=["code|1", "code|1| type"], value_vars=[col for col in df.columns if col.startswith("standard_charge")])
    .filter(~pl.col("variable").is_in(["standard_charge|min", "standard_charge|max", "standard_charge|gross"]))
    .filter(pl.col("value").is_not_null())
    .filter(~pl.col("variable").str.contains("methodology"))
    .filter(pl.col("variable").str.contains("negotiated_dollar"))
    .with_columns([
        pl.col("variable").str.replace("standard_charge|", "").alias("variable"),
        pl.col("value").cast(pl.Float64).alias("negotiated_dollar"),
    ]).with_columns(
        pl.col("variable").str.split_exact("|", 3)
        .struct.rename_fields(["drop", "payer_name", "plan_name"])
        .alias("fields")
    ).unnest("fields")
    .drop(["drop", "variable"])
    .with_columns([
        pl.lit("NYU Long Island").alias("setting"),
        pl.lit("").alias("methodology"),
        pl.col("negotiated_dollar").alias("standard_charge_dollar"),
        ])
    .select(["setting", "payer_name","plan_name","standard_charge_dollar","methodology"])
).write_csv("nyu_92250_cpt.csv")

/var/folders/8_/y69hhps94q5cvrnr_2hlmnq80000gp/T/ipykernel_24823/2263560962.py:1: DeprecationWarning: `DataFrame.melt` is deprecated; use `DataFrame.unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  (df.melt(id_vars=["code|1", "code|1| type"], value_vars=[col for col in df.columns if col.startswith("standard_charge")])
